<a href="https://colab.research.google.com/github/saurabhgupta17888/ADVANCED_NLP/blob/master/Copy_of_GoogleQA_Huggingface_tfhub_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Google Drive and data

In [0]:
from google.colab import drive
drive.mount('/content/drive')
!pip install --upgrade tensorflow
!pip install transformers
 
import os
os.chdir('drive/My Drive/')
 
import tensorflow as tf
import tensorflow_datasets
from transformers import *
print(tf.__version__)
 
import pandas as pd
import numpy as np

KeyboardInterrupt: ignored

In [0]:
train = pd.read_csv("./kaggle/google_quest/train.csv.zip")
test = pd.read_csv("./kaggle/google_quest/test.csv")
sub = pd.read_csv("./kaggle/google_quest/sample_submission.csv")

# Simple Transformers Implementation Example

In [0]:
#!pip install simpletransformers
from simpletransformers.classification import MultiLabelClassificationModel

# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. The first column is the text with type str, and the second column is the label with type int.
train_data = [['Example sentence belonging to class 1', 1], ['Example sentence belonging to class 0', 0]]
train_df = pd.DataFrame(train_data)

eval_data = [['Example eval sentence belonging to class 1', 1], ['Example eval sentence belonging to class 0', 0]]
eval_df = pd.DataFrame(eval_data)

# Create a ClassificationModel
model = ClassificationModel('bert', 'bert-base',use_cuda=False) # You can set class weights by using the optional weight argument

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

Converting to features started.


Running loss: 0.698835
Training of roberta model complete. Saved to outputs/.
Converting to features started.


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


# HuggingFace/Transformers Implementation

## Get BERT embeddings

In [0]:
from transformers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
bert_tokenizer_transformer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [0]:
def _get_segments(sentences):
    sentences_segments = []
    for sent in sentences:
      temp = []
      i = 0
      for token in sent.split(" "):
        temp.append(i)
        if token == "[SEP]":
          i += 1
      sentences_segments.append(temp)
    return sentences_segments

def _get_inputs(df,_maxlen,tokenizer,use_keras_pad=False):


    maxqnans = np.int((_maxlen-20)/2)
    pattern = '[^\w\s]+|\n' # remove everything including newline (|\n) other than words (\w) or spaces (\s)
    
    sentences = ["[CLS] " + " ".join(tokenizer.tokenize(qn)[:maxqnans]) +" [SEP] " 
              + " ".join(tokenizer.tokenize(ans)[:maxqnans]) +" [SEP] " 
              + " ".join(tokenizer.tokenize(title)[:10]) + " [SEP] "
              + " ".join(tokenizer.tokenize(cat)[:10]) +" [SEP]" 
                for (title,qn,ans,cat) 
                in 
              zip(df['question_title'].str.replace(pattern, '').values.tolist(),
              df['question_body'].str.replace(pattern, '').values.tolist(),
              df['answer'].str.replace(pattern, '').values.tolist(),
              df['category'].str.replace(pattern, '').values.tolist())]
              #train.head()[['question_title','question_body','answer','category']].values.tolist()]
    

    #generate masks
    # bert requires a mask for the words which are padded. 
    # Say for example, maxlen is 100, sentence size is 90. then, [1]*90 + [0]*[100-90]
    sentences_mask = [[1]*len(sent.split(" "))+[0]*(_maxlen - len(sent.split(" "))) for sent in sentences]
 
    #generate input ids  
    # if less than max length provided then the words are padded
    if use_keras_pad:
      sentences_padded = pad_sequences(sentences.split(" "), dtype=object, maxlen=10, value='[PAD]',padding='post')
    else:
      sentences_padded = [sent + " [PAD]"*(_maxlen-len(sent.split(" "))) if len(sent.split(" "))!=_maxlen else sent for sent in sentences ]

    sentences_converted = [tokenizer.convert_tokens_to_ids(s.split(" ")) for s in sentences_padded]
    
    #generate segments
    # for each separation [SEP], a new segment is converted
    sentences_segment = _get_segments(sentences_padded)

    genLength = set([len(sent.split(" ")) for sent in sentences_padded])

    if _maxlen < 20:
      raise Exception("max length cannot be less than 20")
    elif len(genLength)!=1: 
      print(genLength)
      raise Exception("sentences are not of same size")



    #convert list into tensor integer arrays and return it
    #return sentences_converted,sentences_segment, sentences_mask
    #return [np.asarray(sentences_converted, dtype=np.int32), 
    #        np.asarray(sentences_segment, dtype=np.int32), 
    #        np.asarray(sentences_mask, dtype=np.int32)]
    return [tf.cast(sentences_converted,tf.int32), tf.cast(sentences_segment,tf.int32), tf.cast(sentences_mask,tf.int32)]

In [0]:
bert_inputs = _get_inputs(df=train.head(),tokenizer=bert_tokenizer_transformer,_maxlen=100)

## Add Pre-trained model using Transformer

In [0]:
#reference: https://github.com/huggingface/transformers

In [0]:
from tensorflow.keras.layers import Dense, Dropout,Embedding, LSTM, Bidirectional, Input, Dropout, GlobalAveragePooling1D
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import sequence

In [0]:
Xtr = bert_inputs
ytr = np.asarray(train.iloc[:5,-30:])

Xte = _get_inputs(test.head(),_maxlen=100, tokenizer = bert_tokenizer_transformer )
yte = np.asarray(test.iloc[:5,-30:])

the pretrained bert model provides pooled output & sequence output. Pooled output is provides at sentence level and is shaped [sentence_count/batch_size, embed_dim] while sequence output provides at word level and is shaped [sentence_count/batch_size, nooftokens, embed_dim]

In [0]:
#reference: https://github.com/huggingface/transformers/issues/1350
MAX_SEQUENCE_LENGTH = 100

token_inputs = Input((MAX_SEQUENCE_LENGTH), dtype=tf.int32, name='input_word_ids')
mask_inputs = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
seg_inputs = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_segments')

bert_model = TFBertModel.from_pretrained("bert-base-uncased")
seq_output,_ = bert_model([token_inputs, mask_inputs, seg_inputs])
X = GlobalAveragePooling1D()(seq_output)
X = Dense(100, activation='relu')(X)
output_= Dense(30, activation='sigmoid', name='output')(X)

bert_model2 = Model([token_inputs, mask_inputs, seg_inputs],output_)
bert_model2.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_segments (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_4 (TFBertModel)   ((None, 100, 768), ( 109482240   input_word_ids[0][0]             
                                                                 input_masks[0][0]          

In [0]:
bert_model2.compile(optimizer='adam', loss='binary_crossentropy')

In [0]:
bert_model2.fit(Xtr,ytr,epochs=1,batch_size = 3)

Train on 5 samples


5/5 [==============================] - 29s 6s/sample - loss: 0.7191


In [0]:
result = bert_model2.predict(Xte)
result.shape

(5, 30)

# Bert TfHub Internet-On Implementation

## Get BERT embeddings

Tfhub implementation of Bert can be found at [Colab notebook by Google Research team](https://colab.research.google.com/github/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb) 
but the current implementation is not compatible for tf2.0. for tf2, use bert-for-tf2
 [package & explanation](https://pypi.org/project/bert-for-tf2/). This notebook uses tf2 implementation

In [0]:
#!pip install bert-for-tf2

In [0]:
import tensorflow_hub as hub
import bert

In [0]:
vocab_file1 = bert_layer.resolved_object.vocab_file.asset_path.numpy()
bert_tokenizer_tfhub = bert.bert_tokenization.FullTokenizer(vocab_file1, do_lower_case=True)

In [0]:
bert_inputs = _get_inputs(df=train.head(),tokenizer=bert_tokenizer_tfhub,_maxlen=100)

## Add Pre-trained model using tfhub bert layer

In [0]:
# very important - use only this bert version in tfhub since its tf2 compatible
# all the instructions are available in the same page
# https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1
bert_path = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"
bert_layer = hub.KerasLayer(bert_path,trainable=True)

In [0]:
_,Xtr_bert= bert_layer(bert_inputs)
ytr = np.asarray(train.iloc[:5,-30:])

In [0]:
from tensorflow.keras.layers import Dense, Dropout,Embedding, LSTM, Bidirectional, Input, Dropout, GlobalAveragePooling1D
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import sequence

In [0]:
def build_model_fullyconnected(MAX_SEQUENCE_LENGTH = 100):
    """add pretrained bert model as a keras layer"""
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    input_segments = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_segments')
    _, sout = bert_layer([input_word_ids, input_masks, input_segments])
    X= Dense(100, activation='relu')(sout) 
    #X= Dense(100, activation='relu')(input_) 
    X = GlobalAveragePooling1D()(X)
    output_= Dense(30, activation='sigmoid', name='output')(X)

    #model = Model(input_,output_)
    model = Model([input_word_ids, input_masks, input_segments],output_)
    print(model.summary())

    return model

def build_model_bertembed(MAX_SEQUENCE_LENGTH = 100):

    input_ = Input(shape = (MAX_SEQUENCE_LENGTH,768), name='bert_enconding')
    X= Dense(100, activation='relu')(input_) 
    X = GlobalAveragePooling1D()(X)
    output_= Dense(30, activation='sigmoid', name='output')(X)
    
    model = Model(input_,output_)
    print(model.summary())
    return model

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_segments (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_masks[0][0]            

In [0]:
model = build_models_fullyconnected()
model.compile(optimizer = "adam",loss = "binary_crossentropy")
history = model.fit(bert_inputs,ytr,epochs=1,batch_size = 3)

model = build_models_bertembed()
model.compile(optimizer = "adam",loss = "binary_crossentropy")
history = model.fit(Xtr_bert,ytr,epochs=1,batch_size = 3)

Train on 5 samples
5/5 [==============================] - 18s 4s/sample - loss: 0.8280
